In [1]:
import pandas as pd
import re

In [2]:
wine = pd.read_csv('data/wine_cleared.csv')
wine = wine.drop('Unnamed: 0', axis=1)
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   country                129908 non-null  object 
 1   description            129971 non-null  object 
 2   designation            129971 non-null  object 
 3   points                 129971 non-null  int64  
 4   price                  129971 non-null  float64
 5   province               129908 non-null  object 
 6   region_1               129971 non-null  object 
 7   taster_name            129971 non-null  object 
 8   taster_twitter_handle  129971 non-null  object 
 9   title                  129971 non-null  object 
 10  variety                129970 non-null  object 
 11  winery                 129971 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 11.9+ MB


 Создадим новый признак price_round, означающий округлённую до целого числа цену за бутылку вина:

In [3]:
# для удобства сразу преобразуем признак в int
wine['price_round'] = wine['price'].round().astype(int)

РАЗБОР ТЕКСТА

Для того, чтобы извлечь из строки число, обозначающее год, необходимо воспользоваться регулярными выражениями.

Регулярные выражения (regexp, или regex) — это механизм для поиска и замены текста. Это шаблоны, которые используются для поиска соответствующей части текста.

Реализация такого механизма существует в pandas в работе со строками. Для того чтобы найти все числа в каждом значении серии, воспользуемся методом str.findall(). Метод возвращает все совпадения с заданным шаблоном в серии pandas. 

In [4]:
regex = '\d{4}' # регулярное выражение для нахождения чисел
wine['year'] = wine['title'].str.findall(regex).str.get(0)

Разберём регулярное выражение \d{4}:

\d — класс символов, обозначает соответствие цифрам в диапазоне цифр [0-9];
{4} в шаблоне означает искать четыре вхождения символа, указанного ранее. В нашем случае это будут четырехзначные числа.

Однако при поиске числа методом data['title'].str.findall(regex) результатом выполнения является список найденных цифр. Поэтому необходимо извлечь первый элемент из списка найденных методом str.get(0), где 0 — первый элемент в списке найденных чисел.

РАЗБОР КАТЕГОРИЙ

В наборе данных винных обзоров самая популярная страна-производитель вина — США. Возможно, это не случайность, и факт производства в США влияет на рейтинг вина. Выделим этот факт.

Вы можете создать новый бинарный признак is_usa и присвоить ему 1 в случае, если вино произведено в США, иначе — 0.

In [5]:
wine['is_usa'] = wine['country'].apply(lambda x: 1 if x == 'US' else 0)

В наборе данных также есть ещё две страны, которые являются не менее популярными производителями вина.

In [6]:
wine['country'].value_counts(sort=True, ascending=False)

country
US                        54504
France                    22093
Italy                     19540
Spain                      6645
Portugal                   5691
Chile                      4472
Argentina                  3800
Austria                    3345
Australia                  2329
Germany                    2165
New Zealand                1419
South Africa               1401
Israel                      505
Greece                      466
Canada                      257
Hungary                     146
Bulgaria                    141
Romania                     120
Uruguay                     109
Turkey                       90
Slovenia                     87
Georgia                      86
England                      74
Croatia                      73
Mexico                       70
Moldova                      59
Brazil                       52
Lebanon                      35
Morocco                      28
Peru                         16
Ukraine                      14


Создайте бинарные признаки is_france, is_italy наподобие признака is_usa.

In [7]:
wine['is_france'] = wine['country'].apply(lambda x: 1 if x == 'France' else 0)
wine['is_italy'] = wine['country'].apply(lambda x: 1 if x == 'Italy' else 0)

In [8]:
wine['is_france'].sum()

22093

In [9]:
wine['is_italy'].sum()

19540

Попробуем вывести правило в определении качества вина: старые вина лучше молодых. В нашем датасете 40 % вин старше 2010 года. 

Создайте новый бинарный признак old_wine, где значение 1 будет обозначать, что вино старше 2010 года.

In [10]:
wine['year'] = pd.to_datetime(wine['year'], errors='coerce')

# для сравнения используем год, заполняем значения признака old_wine, где год вина меньше 2010
wine['old_wine'] = wine['year'].apply(lambda x: 1 if x.year < 2010 else 0) 

In [11]:
wine['old_wine'].sum()

39765

In [12]:
regex = '\((.*?)\)'
wine['locality'] = wine['title'].str.findall(regex).str.get(0)

In [14]:
wine.to_csv('data/wine_cleared_last.csv')